# Prepare script for free_energy_liquid calculations

This Notebook is designed to prepare free_energy calculations basd on finished relax_liquid calculations. 

The actions in this Notebook may get replaced with a built-in buildcombos function later on...


In [1]:
import iprPy
import numpy as np
import atomman as am
import potentials

In [2]:
database_name = 'master'
run_directory_name = 'master_6'

lammps_command = 'C:/Users/lmh1/Documents/LAMMPS/2022-09-15/bin/lmp.exe'

In [3]:
database = iprPy.load_database(database_name)
potdb = potentials.Database(local_database=database, local=True, remote=False)

In [4]:
# Fetch finished relax_dynamic records
parents, parent_df = database.get_records('calculation_relax_liquid', status='finished', return_df=True)

In [5]:
parent_df.keys()

Index(['name', 'key', 'iprPy_version', 'atomman_version', 'script', 'branch',
       'status', 'lammps_version', 'potential_LAMMPS_key',
       'potential_LAMMPS_id', 'potential_key', 'potential_id', 'load_file',
       'load_style', 'load_options', 'parent_key', 'family', 'symbols',
       'composition', 'a_mult1', 'a_mult2', 'b_mult1', 'b_mult2', 'c_mult1',
       'c_mult2', 'atomshift', 'a_uvw', 'b_uvw', 'c_uvw', 'temperature',
       'pressure', 'rdfcutoff', 'volume', 'volume_stderr',
       'measured_temperature', 'measured_temperature_stderr',
       'measured_pressure', 'measured_pressure_stderr', 'E_total',
       'E_total_stderr', 'E_pot', 'E_pot_stderr', 'time_values',
       'msd_x_values', 'msd_y_values', 'msd_z_values', 'msd_values'],
      dtype='object')

In [5]:
# Construct global single keys
prepare_keys = {}
prepare_keys['lammps_command'] = lammps_command
prepare_keys['equilsteps'] = '25000'
prepare_keys['switchsteps'] = '50000'
prepare_keys['sizemults'] = '1 1 1'

# Initialize multi keys
prepare_keys['potential_file'] = []
prepare_keys['potential_content'] = []
prepare_keys['potential_dir'] = []
prepare_keys['potential_dir_content'] = []
prepare_keys['potential_kim_id'] = []
prepare_keys['potential_kim_potid'] = []
prepare_keys['load_file'] = []
prepare_keys['load_content'] = []
prepare_keys['load_style'] = []
prepare_keys['symbols'] = []
prepare_keys['temperature'] = []

for potential_LAMMPS_key in np.unique(parent_df.potential_LAMMPS_key):
    lmppot = potdb.get_lammps_potential(key=potential_LAMMPS_key)
    
    for i in parent_df[parent_df.potential_LAMMPS_key == potential_LAMMPS_key].index:
        parent = parents[i]
        parent_series = parent_df.loc[i]
        
        # Set potential info
        prepare_keys['potential_file'].append(f'{lmppot.name}.json')
        prepare_keys['potential_content'].append(f'record {lmppot.name}')
        if lmppot.pair_style != 'kim':
            prepare_keys['potential_dir'].append(lmppot.name)
            prepare_keys['potential_dir_content'].append(f'tar {lmppot.name}')
            prepare_keys['potential_kim_id'].append('')
            prepare_keys['potential_kim_potid'].append('')
        else:
            prepare_keys['potential_dir'].append('')
            prepare_keys['potential_dir_content'].append('')
            prepare_keys['potential_kim_id'].append(lmppot.id)
            if len(lmppot.potids) > 1:
                prepare_keys['potential_kim_potid'].append(lmppot.potid)
            else:
                prepare_keys['potential_kim_potid'].append('')
                
        # Set load file info
        prepare_keys['load_file'].append(f'{parent.name}/{parent.final_dump["filename"]}')
        prepare_keys['load_content'].append(f'tarfile {parent.name} {parent.final_dump["filename"]}')
        prepare_keys['load_style'].append('atom_dump')
        prepare_keys['symbols'].append(' '.join(parent.final_dump["symbols"]))
        
        prepare_keys['temperature'].append(str(parent.temperature))
    
    


In [6]:
database.prepare(run_directory_name, 'free_energy_liquid', **prepare_keys)

300 existing calculation records found
326 calculation combinations to check
26 new records to prepare


preparing: 100%|#######################################################################| 26/26 [02:25<00:00,  5.60s/it]


In [15]:
for key in prepare_keys:
    print(key, len(prepare_keys[key]))

lammps_command 53
equilsteps 5
switchsteps 5
sizemults 5
potential_file 60
potential_content 60
potential_dir 60
potential_dir_content 60
potential_kim_id 60
potential_kim_potid 60
load_file 60
load_content 60
load_style 60
symbols 60
box_parameters 0
temperature 60
